In [1]:
import pandas as pd
import backtrader as bt
from datetime import datetime
from loguru import logger

In [2]:
class CrossOverTrailPercentStop(bt.Strategy):
    # параметры
    params = dict(
        fast_ma=10,
        slow_ma=20,
        trail_perc_stop=3
    )

    def __init__(self):
        '''рассчёт скользящих средних и индикатора их пересечения'''
        fast_ma = bt.ind.SMA(period=self.p.fast_ma)
        slow_ma = bt.ind.SMA(period=self.p.slow_ma)
        self.crossover = bt.ind.CrossOver(fast_ma, slow_ma)
        
        self.orderl = list() # для хранения состояния заявки

    def notify_order(self, order):
        '''вывод сделок в консоль'''
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                logger.info(f'BUY@price {order.executed.price}')
            elif order.issell():
                logger.info(f'SELL@price {order.executed.price}')
        elif order.status in [order.Margin, order.Rejected]:
            logger.warning('Order Margin/Rejected')

        self.order = None

    def next(self):
        if not self.position and self.crossover > 0:
                o1 = self.buy(transmit=False) # основная зявки
                o2 = self.sell(exectype=bt.Order.StopTrail, trailpercent=self.p.trail_perc_stop/100, parent=o1, transmit=True)
                self.orderl = [o1, o2]
        elif self.crossover < 0:
            for o in self.orderl: 
                self.cancel(o)
            self.close()


In [3]:
cerebro = bt.Cerebro()
commission = 0.05
cerebro.broker.setcommission(commission=commission/100, leverage=1)
cerebro.broker.setcash(10000.) # стартовый кэш
cerebro.addsizer(bt.sizers.PercentSizer, percents=95) # размер позиции

df = pd.read_csv(
    '..//data//1d//AAPL.csv', # файл с котировками
    index_col=0,
    parse_dates=True
)

data = bt.feeds.PandasData(
    dataname=df,
    openinterest=-1,
    fromdate=datetime(2005, 1, 1), # стартовая дата
    # todate=datetime(2005, 2, 1),
    timeframe=bt.TimeFrame.Days,
    compression=1
)

cerebro.adddata(data) # добавляем котировки
cerebro.addstrategy(CrossOverTrailPercentStop,fast_ma=10,slow_ma=20,trail_perc_stop=3) # добавляем стратегию
cerebro.run() # запускаем рассчёты

2021-10-20 13:09:52.310 | INFO     | __main__:notify_order:24 - BUY@price 1.5199999809265134
2021-10-20 13:09:52.314 | INFO     | __main__:notify_order:26 - SELL@price 1.4827138793468475
2021-10-20 13:09:52.333 | INFO     | __main__:notify_order:24 - BUY@price 1.2649999856948853
2021-10-20 13:09:52.337 | INFO     | __main__:notify_order:26 - SELL@price 1.2214289903640747
2021-10-20 13:09:52.344 | INFO     | __main__:notify_order:24 - BUY@price 1.410714030265808
2021-10-20 13:09:52.351 | INFO     | __main__:notify_order:26 - SELL@price 1.362856984138489
2021-10-20 13:09:52.372 | INFO     | __main__:notify_order:24 - BUY@price 1.3296430110931396
2021-10-20 13:09:52.378 | INFO     | __main__:notify_order:26 - SELL@price 1.3146430253982544
2021-10-20 13:09:52.392 | INFO     | __main__:notify_order:24 - BUY@price 1.4632140398025513
2021-10-20 13:09:52.401 | INFO     | __main__:notify_order:26 - SELL@price 1.5242861437797546
2021-10-20 13:09:52.411 | INFO     | __main__:notify_order:24 - BUY

In [4]:
cerebro.plot(iplot=False) # рисуем графики

[[<Figure size 640x480 with 5 Axes>]]